In [ ]:
from extract import YouTubeExtractor
from config import config


fg_store = YouTubeExtractor('@5Gstore', config)
uploads_playlist_id = fg_store.get_uploads_playlist_id()
print(uploads_playlist_id)


In [ ]:

fg_store.fetch_videos_with_transcripts()


In [ ]:

# mirc = YouTubeExtractor('@MobileInternetResourceCenter')
# frontier_us = YouTubeExtractor('@Frontierus')
# mobile_must_have = YouTubeExtractor('@MobileMustHave')
# techno_rv = YouTubeExtractor('@Technorv')
